In [2]:
import cv2
import os
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from absl import app, flags, logging
from absl.flags import FLAGS

from models import RetinaFaceModel
from utils import (set_memory_growth, load_yaml,
                           pad_input_image, recover_pad_output)

In [3]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
set_memory_growth()

cfg = load_yaml('retinaface_res50.yaml')

# define network
model = RetinaFaceModel(cfg, training=False, iou_th=0.4, score_th=0.5)

# load checkpoint
checkpoint_dir = cfg['sub_name']
checkpoint = tf.train.Checkpoint(model=model)
if tf.train.latest_checkpoint(checkpoint_dir):
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    print("[*] load ckpt from {}.".format(
        tf.train.latest_checkpoint(checkpoint_dir)))
else:
    print("[*] Cannot find ckpt from {}.".format(checkpoint_dir))
    exit()


[*] load ckpt from retinaface_res50\ckpt-81.


In [4]:
def draw_bbox_landm(img, ann, img_height, img_width):
    """draw bboxes and landmarks"""
    # bbox
    x1, y1, x2, y2 = int(ann[0] * img_width), int(ann[1] * img_height), \
                     int(ann[2] * img_width), int(ann[3] * img_height)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # confidence
    text = "{:.4f}".format(ann[15])
    cv2.putText(img, text, (int(ann[0] * img_width), int(ann[1] * img_height)),
                cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

    # landmark
    if ann[14] > 0:
        cv2.circle(img, (int(ann[4] * img_width),
                         int(ann[5] * img_height)), 1, (255, 255, 0), 2)
        cv2.circle(img, (int(ann[6] * img_width),
                         int(ann[7] * img_height)), 1, (0, 255, 255), 2)
        cv2.circle(img, (int(ann[8] * img_width),
                         int(ann[9] * img_height)), 1, (255, 0, 0), 2)
        cv2.circle(img, (int(ann[10] * img_width),
                         int(ann[11] * img_height)), 1, (0, 100, 255), 2)
        cv2.circle(img, (int(ann[12] * img_width),
                         int(ann[13] * img_height)), 1, (255, 0, 100), 2)

In [5]:
img_raw = cv2.imread('E:\\ML\\Deepfake Challenge Kaggle\\output_frames\\aettqgevhz.mp4_FAKE\\00010.jpg')
img_height_raw, img_width_raw, _ = img_raw.shape
img = np.float32(img_raw.copy())

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# pad input image to avoid unmatched shape problem
img, pad_params = pad_input_image(img, max_steps=max(cfg['steps']))

# run model
outputs = model(img[np.newaxis, ...]).numpy()

# recover padding effect
outputs = recover_pad_output(outputs, pad_params)

for prior_index in range(len(outputs)):
        draw_bbox_landm(img_raw, outputs[prior_index], img_height_raw,
                        img_width_raw)
        plt.figure()
        plt.imshow(img_raw)
        plt.plot()

ResourceExhaustedError: OOM when allocating tensor with shape[1,272,480,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:BiasAdd] name: RetinaFaceModel/ResNet50_extrator/conv2_block1_0_conv/BiasAdd/

In [11]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16940211446171870528, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1444338073
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7563432922746003154
 physical_device_desc: "device: 0, name: GeForce 940MX, pci bus id: 0000:01:00.0, compute capability: 5.0"]